In [1]:
print ('TFM: RISK MANAGEMENT IN IBERIAN SPOT MARKET.')
    

TFM: RISK MANAGEMENT IN IBERIAN SPOT MARKET.


In [3]:
import pandas as pd
import requests # When you ping a website or portal for information this is called making a request.
import json # JavaScript Object Notation. Storing and Exanging files. 
import os # You can find important information about your location or about the process. 
import win32com.client as win32
%pylab inline 
# Import numpy as np and import matploltlib(plt as pyplot)
import esios # Imports our esios module.
os.getcwd()  #Current working directory.

Populating the interactive namespace from numpy and matplotlib


'C:\\Users\\ilopez18'

In [4]:
os.chdir('K:\Operacion\Informes de operación diarios\Descargas API esios\\') # working directory. esios.py must be there.

In [5]:
esios.header() #  Header API request.

# In esios.py module introduce your own TOKEN.

{'Accept': 'application/json; application/vnd.esios-api-v1+json',
 'Authorization': 'Token token="1111111111111111111111111111111111"',
 'Content-Type': 'application/json',
 'Cookie': '',
 'Host': 'api.esios.ree.es'}

In [6]:
esios_ind = esios.indicators_dict()
esios_ind

{'Energía asignada en Mercado Intradiario Continuo': 1442,
 'Generación programada PHFC UGH + no UGH': 10271,
 'Generación programada PHFC total': 10287,
 'Generación programada + Consumo bombeo + Cable Baleares PHFC': 10286,
 'Ajuste de programas PHFC': 10285,
 'Saldo total interconexiones programa PHFC': 10284,
 'Generación programada PHFC otras renovables': 10283,
 'Generación programada PHFC Carbón': 10282,
 'Generación programada PHFC Eólica': 10281,
 'Generación programada PHFC Anexo II RD 134/2010': 10280,
 'Demanda programada PHFC total': 10279,
 'Generación programada PHFC Saldo Andorra': 10278,
 'Generación programada PHFC Saldo Marruecos': 10277,
 'Generación programada PHFC Cogeneración': 10273,
 'Generación programada PHFC Saldo Portugal': 10276,
 'Generación programada PHFC Saldo Francia': 10275,
 'Generación programada PHFC Residuos': 10274,
 'Generación programada PHFC Fuel-Gas': 10272,
 'Generación programada PHFC Importación Andorra': 1430,
 'Generación programada PHF

In [7]:
len(esios_ind)# Number or indicators.

1598

In [8]:
#Sort esios_ind diccionario by key, giving a list. Introducing a 'x[0]' will sort by value.
sorted_esios_ind = sorted(esios_ind.items(), key=lambda x: x[1])
sorted_esios_ind

[('Generación programada PBF Hidráulica UGH', 1),
 ('Generación programada PBF Hidráulica no UGH', 2),
 ('Generación programada PBF Turbinación bombeo', 3),
 ('Generación programada PBF Nuclear', 4),
 ('Generación programada PBF Hulla antracita Anexo II RD 134/2010', 5),
 ('Generación programada PBF Hulla sub-bituminosa Anexo II RD 134/2010', 6),
 ('Generación programada PBF Hulla antracita', 7),
 ('Generación programada PBF Hulla sub-bituminosa', 8),
 ('Generación programada PBF Ciclo combinado', 9),
 ('Generación programada PBF Fuel', 10),
 ('Generación programada PBF Gas Natural', 11),
 ('Generación programada PBF Eólica terrestre', 12),
 ('Generación programada PBF Eólica marina', 13),
 ('Generación programada PBF Solar fotovoltaica', 14),
 ('Generación programada PBF Solar térmica', 15),
 ('Generación programada PBF Oceano y geotérmica', 16),
 ('Generación programada PBF Gas Natural Cogeneración', 17),
 ('Generación programada PBF Derivados del petróleo ó carbón', 18),
 ('Generaci

In [9]:
#esios.ups()

In [10]:
#Alternative way to download "UNIDADES DE PROGRAMACION" dataset.
path ="C:/Users/ilopez18/Desktop/Data.csv" #Download "UNIDADES DE PORGRAMACION" from www.esios.ree.es, rename it as Data.
programunits = pd.read_csv(path, sep=';')
programunits.head()

,Código de UP,Descripción corta,Descripción larga,Potencia máxima MW,Código EIC,Tipo de producción,Negocio,Zona de Regulación,Sujeto del Mercado,Tipo de UP
0,A12PAME,A12PAME,A12PAME,"0,5",18WA12PAME-123-U,Derivados del petróleo ó carbón,Venta,SZR,AME,Generación
1,A2GAME,A2GAME,A2GAME,"3,8",18WA2GAME-1234-E,Energía residual,Venta,SZR,AME,Generación
2,ABA1,ABA1,ABARAN,"0,3",18WABA1-123456-K,Hidráulica UGH,Venta,SZR,BARER,Generación
3,ABA2,ABA2,ABARAN 2,"0,4",18WABA2-123456-A,Hidráulica UGH,Venta,SZR,BARER,Generación
4,ABO1,ABOÑO 1,C.T. ABOÑO 1,"341,7",18WABO1-12345-0X,Hulla antracita,Venta,HC,HC G,Generación


In [11]:
#Columns tittles.
programunits.columns.values.tolist()


['Código de UP',
 'Descripción corta',
 'Descripción larga',
 'Potencia máxima MW',
 'Código EIC',
 'Tipo de producción',
 'Negocio',
 'Zona de Regulación',
 'Sujeto del Mercado',
 'Tipo de UP']

In [12]:
#Erase spaces from dataframe head.
programunits.columns = programunits.columns.str.replace(' ','')
programunits

,CódigodeUP,Descripcióncorta,Descripciónlarga,PotenciamáximaMW,CódigoEIC,Tipodeproducción,Negocio,ZonadeRegulación,SujetodelMercado,TipodeUP
0,A12PAME,A12PAME,A12PAME,"0,5",18WA12PAME-123-U,Derivados del petróleo ó carbón,Venta,SZR,AME,Generación
1,A2GAME,A2GAME,A2GAME,"3,8",18WA2GAME-1234-E,Energía residual,Venta,SZR,AME,Generación
2,ABA1,ABA1,ABARAN,"0,3",18WABA1-123456-K,Hidráulica UGH,Venta,SZR,BARER,Generación
3,ABA2,ABA2,ABARAN 2,"0,4",18WABA2-123456-A,Hidráulica UGH,Venta,SZR,BARER,Generación
4,ABO1,ABOÑO 1,C.T. ABOÑO 1,"341,7",18WABO1-12345-0X,Hulla antracita,Venta,HC,HC G,Generación
5,ABO2,ABOÑO 2,C.T. ABOÑO 2,"535,8",18WABO2-12345-0N,Hulla antracita,Venta,HC,HC G,Generación
6,ABOUC01,ABOUTWHI,"ABOUT WHITE, S.L.","0,1",18WABOUC01-123-F,Comercializadores mercado libre,Compra,SZR,ABOUT,Comercializadores
7,ACBFRAE,ACBFRA EXP,ACCIÓN COORDINADA DE BALANCE ES A FR,"9.999,00",18WACBFRAE-12--2,Unidad instrumental internacional,Compra,SZR,REE,Unidad instrumental internacional
8,ACBFRAI,ACBFRA IMP,ACCIÓN COORDINADA DE BALANCE FR A ES,"9.999,00",18WACBFRAI-12--B,Unidad instrumental internacional,Venta,SZR,REE,Unidad instrumental internacional
9,ACBPORE,ACBPOR EXP,ACCIÓN COORDINADA DE BALANCE ES A PT,"9.999,00",18WACBPORE-123-3,Unidad instrumental internacional,Compra,SZR,REE,Unidad instrumental internacional


In [13]:
#Is there any NAN?
programunits.isnull().values.any()

True

In [14]:
#Where are this NaN.
programunits.isnull().sum()

CódigodeUP           0
Descripcióncorta     0
Descripciónlarga     0
PotenciamáximaMW     0
CódigoEIC            0
Tipodeproducción    14
Negocio              0
ZonadeRegulación     0
SujetodelMercado     0
TipodeUP             0
dtype: int64

In [15]:
#DISPLAY dataframe cointaining NAN.
programunitsNAN = programunits[programunits.isnull().any(axis=1)]
programunitsNAN

,CódigodeUP,Descripcióncorta,Descripciónlarga,PotenciamáximaMW,CódigoEIC,Tipodeproducción,Negocio,ZonadeRegulación,SujetodelMercado,TipodeUP
380,EGLPCX1,EGLPCX1,PORFOLIO COMPRA GENERAC,"9.999,00",18W000000000ABZD,NaN,Compra,SZR,EGLE,Unidades porfolio
381,EGLPVX1,EGLPVX1,PORFOLIO VENTA GENER,"9.999,00",18W000000000AC06,NaN,Venta,SZR,EGLE,Unidades porfolio
749,GASPCX1,GASPCX1,GASN PORTFOLIO COMPRA,"9.999,00",18W000000000AAAT,NaN,Compra,SZR,GASN,Unidades porfolio
750,GASPVX1,GASPVX1,GASN PORTFOLIO VENTA,"9.999,00",18W000000000AABR,NaN,Venta,SZR,GASN,Unidades porfolio
817,GNRPCX1,GNRPCX1,GNRPCX1,"9.999,00",18W000000000AGE3,NaN,Compra,SZR,GNRA,Unidades porfolio
818,GNRPVX1,GNRPVX1,GNRPVX1,"9.999,00",18W000000000AGF1,NaN,Venta,SZR,GNRA,Unidades porfolio
1030,IBGCCX1,IBGCCX1,UNID PORF IBGES COMERC COMPRA,"9.999,00",18W000000000A7JK,NaN,Compra,SZR,IBGES,Unidades porfolio
1031,IBGCVX1,IBGCVX1,UNID PORF IBGES COMERC VENT,"9.999,00",18W000000000A7IM,NaN,Venta,SZR,IBGES,Unidades porfolio
1035,IBGPCX1,IBGPCX1,PROD PORF IBGES COMERC COMPR,"9.999,00",18W000000000A7KI,NaN,Compra,SZR,IBGES,Unidades porfolio
1036,IBGPVX1,IBGPVX1,PROD PORF IBGES COMERC VENTA,"9.999,00",18W000000000A7LG,NaN,Venta,SZR,IBGES,Unidades porfolio


In [16]:
#change NAN for Porfolio
programunits.fillna('PORFOLIO', inplace=True)
programunits

,CódigodeUP,Descripcióncorta,Descripciónlarga,PotenciamáximaMW,CódigoEIC,Tipodeproducción,Negocio,ZonadeRegulación,SujetodelMercado,TipodeUP
0,A12PAME,A12PAME,A12PAME,"0,5",18WA12PAME-123-U,Derivados del petróleo ó carbón,Venta,SZR,AME,Generación
1,A2GAME,A2GAME,A2GAME,"3,8",18WA2GAME-1234-E,Energía residual,Venta,SZR,AME,Generación
2,ABA1,ABA1,ABARAN,"0,3",18WABA1-123456-K,Hidráulica UGH,Venta,SZR,BARER,Generación
3,ABA2,ABA2,ABARAN 2,"0,4",18WABA2-123456-A,Hidráulica UGH,Venta,SZR,BARER,Generación
4,ABO1,ABOÑO 1,C.T. ABOÑO 1,"341,7",18WABO1-12345-0X,Hulla antracita,Venta,HC,HC G,Generación
5,ABO2,ABOÑO 2,C.T. ABOÑO 2,"535,8",18WABO2-12345-0N,Hulla antracita,Venta,HC,HC G,Generación
6,ABOUC01,ABOUTWHI,"ABOUT WHITE, S.L.","0,1",18WABOUC01-123-F,Comercializadores mercado libre,Compra,SZR,ABOUT,Comercializadores
7,ACBFRAE,ACBFRA EXP,ACCIÓN COORDINADA DE BALANCE ES A FR,"9.999,00",18WACBFRAE-12--2,Unidad instrumental internacional,Compra,SZR,REE,Unidad instrumental internacional
8,ACBFRAI,ACBFRA IMP,ACCIÓN COORDINADA DE BALANCE FR A ES,"9.999,00",18WACBFRAI-12--B,Unidad instrumental internacional,Venta,SZR,REE,Unidad instrumental internacional
9,ACBPORE,ACBPOR EXP,ACCIÓN COORDINADA DE BALANCE ES A PT,"9.999,00",18WACBPORE-123-3,Unidad instrumental internacional,Compra,SZR,REE,Unidad instrumental internacional


In [19]:
#Unique values in column 'TiposdeProduccion'
programunits.Tipodeproducción.unique()


array(['Derivados del petróleo ó carbón', 'Energía residual',
       'Hidráulica UGH', 'Hulla antracita',
       'Comercializadores mercado libre',
       'Unidad instrumental internacional', 'Ciclo Combinado',
       'Consumos directos en mercado', 'Solar fotovoltaica',
       'Consumo bombeo', 'Turbinación bombeo', 'Nuclear',
       'Consumo de Servicios Auxiliares', 'Exportación Francia',
       'Importación Francia', 'Gas Natural Cogeneración',
       'Hulla sub-bituminosa', 'Biogás', 'Residuos domésticos y similares',
       'Eólica terrestre', 'Comercializadores último recurso',
       'Enlace Baleares', 'Fuel', 'PORFOLIO', 'Exportación Marruecos',
       'Exportación Andorra', 'Importación Marruecos', 'Hidráulica no UGH',
       'Genéricas', 'Oceano y geotérmica', 'Saldo acoplamiento mercado',
       'Biomasa', 'Residuos varios', 'Solar térmica',
       'Subproductos minería'], dtype=object)

In [39]:
indicadores_evcd  = ['Derechos de Cobro del proceso de Gestión de Desvíos',
                      'Derechos de Cobro del proceso de Regulación de Terciaria',
                      'Derechos de Cobro por energía de Regulación Secundaria utilizada',
                      'Obligaciones de Pago del proceso de Gestión de Desvíos',
                      'Obligaciones de Pago del proceso de Regulación de Terciaria',
                      'Obligaciones de Pago por energía de Regulación Secundaria utilizada',
                      'Coste de solución de Restricciones técnicas diario',
                      'Coste del proceso de solución de Restricciones Técnicas de Seguridad en Tiempo Real',
                      'Coste de Reserva de Potencia Adicional a Subir',
                      'Coste de banda de Regulación Secundaria']


indicadores_pas = ['Precio medio horario componente reserva de potencia adicional a subir ',
                      'Asignación Reserva de potencia adicional a subir']


informes_diarios = ['Precio medio restricciones técnicas diario Fase II bajar',
                      'Precio Banda de regulación secundaria',
                      'Precio de los desvíos medidos a subir',
                      'Precio de los desvíos medidos a bajar',
                      'Precio Reserva de potencia adicional a subir',
                      'Asignación Reserva de potencia adicional a subir',
                      'Precio de Regulación Secundaria bajar',
                      'Precio de Regulación Secundaria subir']

nombres = [ind.replace(' ','_') for ind in indicadores_evcd]

In [40]:

#INDICADORES DE DATOS
header = {'Accept': 'application/json; application/vnd.esios-api-v1+json',\
     'Content-Type': 'application/json',\
     'Host': 'api.esios.ree.es',\
     'Authorization': 'Token token="0a3ec68a89579911bd7610e20df0f79c83f79b10275b19025aaf414136c91f9a"',\
     'Cookie':'' }
r = requests.get('https://api.esios.ree.es/indicators', headers=header).json()#,params=param)

In [41]:
ind = r['indicators']
indicators = {item['name']:item['id'] for item in ind}
indicators

{'Energía asignada en Mercado Intradiario Continuo': 1442,
 'Generación programada PHFC UGH + no UGH': 10271,
 'Generación programada PHFC total': 10287,
 'Generación programada + Consumo bombeo + Cable Baleares PHFC': 10286,
 'Ajuste de programas PHFC': 10285,
 'Saldo total interconexiones programa PHFC': 10284,
 'Generación programada PHFC otras renovables': 10283,
 'Generación programada PHFC Carbón': 10282,
 'Generación programada PHFC Eólica': 10281,
 'Generación programada PHFC Anexo II RD 134/2010': 10280,
 'Demanda programada PHFC total': 10279,
 'Generación programada PHFC Saldo Andorra': 10278,
 'Generación programada PHFC Saldo Marruecos': 10277,
 'Generación programada PHFC Cogeneración': 10273,
 'Generación programada PHFC Saldo Portugal': 10276,
 'Generación programada PHFC Saldo Francia': 10275,
 'Generación programada PHFC Residuos': 10274,
 'Generación programada PHFC Fuel-Gas': 10272,
 'Generación programada PHFC Importación Andorra': 1430,
 'Generación programada PHF

In [42]:
# FUNCIONES DE FECHA
from calendar import monthrange
import datetime as dt
from datetime import datetime
now = dt.datetime.today()
dia = now.day
mes = now.month   
anho = now.year
 #>>> datetime.datetime(2017, 6, 14, 11, 25, 12, 11273)

fechab_ini_mes = '01-' + str(mes) + '-' + str(anho)
fechab_ayer = str(dia-1) + '-' + str(mes) + '-' + str(anho)
fechab_hoy = str(dia+1) + '-' + str(mes) + '-' + str(anho)

if len(str(mes))==1:
    mes = '0'+str(mes)
else:
    mes = str(mes)

strMes = str(now.year)+mes


now.month
fechab_ini_mes, fechab_ayer

print(dia,mes,anho)

30 08 2018


In [43]:

#ARCHIVOS PARA RELLENAR EVOLUCION COSTE DESVIO
header = {'Accept': 'application/json; application/vnd.esios-api-v1+json',\
     'Content-Type': 'application/json',\
     'Host': 'api.esios.ree.es',\
     'Authorization': 'Token token="0a3ec68a89579911bd7610e20df0f79c83f79b10275b19025aaf414136c91f9a"',\
     'Cookie':''}

# param = {'start_date':fechab_ini_mes,'end_date':fechab_ayer,'time_trunc':'month'}
param = {'start_date':'01-07-2018','end_date':'30-07-2018','time_trunc':'month'}

val = []

ruta = 'K:\Operacion\Informes de operación diarios\Descargas API esios\\'\
            + 'evolucion_coste_desvio' + '_' + strMes + '.csv'

#for i in enumerate(indicadores_evcd):

for i in range(len(indicadores_evcd)):
    url = 'https://api.esios.ree.es/indicators/' + str(indicators[indicadores_evcd[i]])

    r = requests.get(url, headers=header,params=param).json()
    print(r)
    valores = r['indicator']
    list_val = valores['values']
    
    if list_val:
        val.append(list_val[0]['value'])
    else:
        val.append('NA')
    #val = [round(x,1) for x in val]
    
df = pd.DataFrame({'concepto':indicadores_evcd,
    'valores':val,
    'mes':now.month})

df['valores'] = (df['valores'].replace('\.','', regex=True)
                        .replace(',','.', regex=True)
                        .astype(float)/1e6)
#df['valores'] = df['valores']/1e6

#df = df.groupby('Derechos de Cobro del proceso de Regulación de Terciaria','Obligaciones de Pago del proceso de Regulación de Terciaria')['valores'].agg(['sum','count'])
#solo para columnas?

df2 = df.assign(sumita = ['Ter',df['valores'][1]/1e6+df['valores'][4]/1e6,'Sec',df['valores'][2]/1e6+df['valores'][5]/1e6,'Des',df['valores'][0]/1e6+df['valores'][3]/1e6,0,0,0,0])
print (df2)


df2.to_csv(ruta, sep=';',decimal=',')

{'indicator': {'name': 'Derechos de Cobro del proceso de Gestión de Desvíos', 'short_name': 'Gestión de Desvíos', 'id': 714, 'composited': False, 'step_type': 'step', 'disaggregated': False, 'magnitud': [{'name': 'Coste', 'id': 24}], 'tiempo': [{'name': 'Hora', 'id': 4}], 'geos': [{'geo_id': 8741, 'geo_name': 'Península'}], 'values_updated_at': '2018-07-30T07:59:55.000+02:00', 'values': [{'value': 14325087.04, 'datetime': '2018-07-01T00:00:00.000+02:00', 'datetime_utc': '2018-06-30T22:00:00Z', 'tz_time': '2018-06-30T22:00:00.000Z', 'geo_id': 8741, 'geo_name': 'Península'}]}}
{'indicator': {'name': 'Derechos de Cobro del proceso de Regulación de Terciaria', 'short_name': 'Regulación Terciaria', 'id': 716, 'composited': False, 'step_type': 'step', 'disaggregated': False, 'magnitud': [{'name': 'Coste', 'id': 24}], 'tiempo': [{'name': 'Hora', 'id': 4}], 'geos': [{'geo_id': 8741, 'geo_name': 'Península'}], 'values_updated_at': '2018-07-31T07:56:21.000+02:00', 'values': [{'value': 7939051.75

In [44]:
#ARCHIVOS INFORMES DIARIOS
header = {'Accept': 'application/json; application/vnd.esios-api-v1+json',\
     'Content-Type': 'application/json',\
     'Host': 'api.esios.ree.es',\
     'Authorization': 'Token token="0a3ec68a89579911bd7610e20df0f79c83f79b10275b19025aaf414136c91f9a"',\
     'Cookie':'' }

param = {'start_date':fechab_ini_mes,'end_date':fechab_hoy,'time_trunc':'hour'}

for ind in informes_diarios:
    url = 'https://api.esios.ree.es/indicators/' + str(indicators[ind])
    ruta = 'K:\Operacion\Informes de operación diarios\Descargas API esios\\'\
            + ind.replace(' ','_') + '_' + strMes +  '.csv'
    r = requests.get(url, headers=header,params=param).json()
    valores = r['indicator']
    list_val = valores['values']
    df = pd.DataFrame(list_val)
    df['fecha'] = df['datetime'].str[:10]
    df.to_csv(ruta, sep=';',decimal=',')

In [23]:
esios.ups()

KeyError: 'UnidadesProgramacion'